### Библиотеки

In [113]:
import numpy as np
import pandas as pd
import json
import re
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from IPython.display import IFrame
import geocoder
import copy

### Функции

In [197]:
def find_closest(data:list,my_loc=False, dir_loc=False, by_coords=False, coords=[]) -> (dict, list, str):
    '''
    Поиск ближайшего города
    Параметры:
        data:list - список с информацией по субъектам,
        my_loc:bool - флаг собственной локации,
        dir_loc:bool - флаг заданной локации,
        by_coords:bool - флаг заданной локации координатами,
        coords:list - координаты локации
    Возвращает:
        dict - словарь с информацией по подходящему субъекту
    '''
    
    if my_loc and not coords:
        print('Собственная локация работает не очень стабильно.')
        geo = geocoder.ip('me')
        geo = geo.latlng
        loc_name = 'Ваша локация'
    elif dir_loc and not coords:
        adres = str(input('Введите адрес на английском языке (город или точное местоположение). '))
        geo = geolocator.geocode(adres)
        geo = [geo.latitude, geo.longitude]
        print(geo)
        loc_name = adres
    elif by_coords or coords:
        geo = coords
        loc_name = str(coords)
    else:
        return 'Внимание ошибка! Определитесь с точкой расчета.'

    target = 10**5
    target_sub = []
    for sub in data:
        distance = count_distance(geo[0], geo[1], sub['subjects_board_cities_coords'][0], sub['subjects_board_cities_coords'][1])
        if target > distance:
            target = distance
            target_sub = sub
            
    return(target_sub, geo, loc_name)


def count_distance(lat1:float, lon1:float, lat2:float, lon2:float) -> float:
    '''
    Расчет расстояния по формуле гаверсинуса
    Параметры:
        lat1:float - широта 1-ой точки
        lon1:float - долгота 1-ой точки
        lat2:float - широта 2-ой точки
        lon2:float - долгота 2-ой точки
    Возвращает:
        float - расстояние между точками
    '''
    # p — коэффициент преобразования угла, выраженного в градусах, в радианы: π/180.
    p = 0.017453292519943295
    hav = 0.5 - np.cos((lat2-lat1)*p)/2 + np.cos(lat1*p)*np.cos(lat2*p) * (1-np.cos((lon2-lon1)*p)) / 2
    return 12742 * np.arcsin(np.sqrt(hav))

def coord_fixer(coords:str) -> list:
    '''
    Необходима для конвертации записаных строковых координат
    городов в кортеж
    Параметры:
        coords:str - координаты города
    Возвращает:
        set - преобразованный кортеж координат
    '''
    regex = r'[0-9]+,[0-9]+'
    coords = re.findall(regex,coords)
    coords = list(map(lambda x: float(x.replace(',','.')), coords))
    return coords

def embed_map(m:folium.folium.Map, file_name:str) -> IFrame:
    '''
    Html настройка реальной карты
    Параметры:
        m:folium.folium.Map - folium карта
        file_name:str - название карты
    Возвращает:
        IFrame
    '''
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

### Считывание данных из json-файла

In [147]:
with open('rusubjects_data.json', 'r') as openfile:
    data = json.load(openfile)

In [148]:
data[:3]

[{'subjects_board': 'Республика Карелия',
  'board_countries': ['Финляндия'],
  'subjects_board_cities': 'Петрозаводск',
  'subjects_board_cities_coords': '61,7849 с. ш., 34,3469 в. д.'},
 {'subjects_board': 'Калининградская область',
  'board_countries': ['Польша', 'Литва'],
  'subjects_board_cities': 'Калининград',
  'subjects_board_cities_coords': '54,7065 с. ш., 20,511 в. д.'},
 {'subjects_board': 'Ленинградская область',
  'board_countries': ['Финляндия', 'Эстония'],
  'subjects_board_cities': 'Санкт-Петербург',
  'subjects_board_cities_coords': '59,9386 с. ш., 30,3141 в. д.'}]

### Поиск ближайшего города/субъекта

In [201]:
closest_sub, geo, loc_name = find_closest(data_cf, dir_loc=True)
closest_sub

Введите адрес на английском языке (город или точное местоположение). Norilsk
[69.3498394, 88.200517]


{'subjects_board': 'Новосибирская область',
 'board_countries': ['Казахстан'],
 'subjects_board_cities': 'Новосибирск',
 'subjects_board_cities_coords': [55.0415, 82.9346]}

### Подготовка датасета

In [202]:
data_cf = copy.deepcopy(data)
for sub in data_cf:
    sub['subjects_board_cities_coords'] = coord_fixer(sub['subjects_board_cities_coords'])

### Построение карты

In [203]:
rus_map = folium.Map(
          location=[64.0914, 101.6016],
          zoom_start=3
)

folium.Marker(
    location=geo,
    tooltip=loc_name,
    icon=folium.Icon(icon="glyphicon glyphicon-hand-down", color='red', icon_color='white'),
).add_to(rus_map)

for sub in data_cf:
    if sub['subjects_board'] == closest_sub['subjects_board']:
        folium.Marker(
            location=sub['subjects_board_cities_coords'],
            tooltip=sub['subjects_board_cities'],
            icon=folium.Icon(icon="glyphicon glyphicon-bookmark", color='red', icon_color='white'),
        ).add_to(rus_map)
        
    else:
        
        folium.Marker(
            location=sub['subjects_board_cities_coords'],
            tooltip=sub['subjects_board_cities'],
            icon=folium.Icon(icon="glyphicon glyphicon-menu-down", color='blue', icon_color='red'),
        ).add_to(rus_map)
        
folium.PolyLine([geo, closest_sub['subjects_board_cities_coords']], tooltip="Coast", color='red').add_to(rus_map)

embed_map(rus_map, 'rus_map.html')